## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [1]:
from datetime import datetime
import json
from observation import Sdataset, Analysis
import pandas as pd

def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

def add_nbre_pdc(data, name='ok', pr=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    #data[name] = data[name] & data['nbre_pdc_ok']
    if pr:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [6]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23']
date= dates[-1]
log = {'date_irve': date, 
       'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
#      'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  33382


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [7]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 1 % des points de charge sont hors itinerance

In [8]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  1697
nombre de pdc en itinerance   :  31685


In [9]:
# arborescence des champs et nombre de valeurs différentes
il = Sdataset(itinerance_init)
print(il.tree())

-1: root-derived (31685)
   0 : index (31685)
      1 : contact_operateur (128)
      2 : nom_enseigne (2326)
      3 : coordonneesXY (11031)
      4 : adresse_station (10633)
      5 : id_station_itinerance (15801)
      6 : nom_station (11237)
      7 : implantation_station (5)
      8 : nbre_pdc (28)
      9 : condition_acces (2)
      10: horaires (225)
      12: id_pdc_itinerance (30950)
         11: station_deux_roues (6)
      13: date_maj (545)
      14: last_modified (477)


-----------------------
## 4 - Bilan initial intégrité
- plus de 50 % des lignes présentent un défaut d'intégrité

In [10]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           1468
contact_operateur - id_station_itinerance             15
nom_enseigne - id_station_itinerance                1224
coordonneesXY - id_station_itinerance               1196
id_station_itinerance - id_pdc_itinerance            966
nom_station - id_station_itinerance                 1201
implantation_station - id_station_itinerance         482
nbre_pdc - id_station_itinerance                     800
condition_acces - id_station_itinerance               62
horaires - id_station_itinerance                      60
station_deux_roues - id_station_itinerance           454
adresse_station - coordonneesXY                     1212

nombre d'enregistrements sans erreurs :  26877
nombre d'enregistrements avec au moins une erreur :  4808
taux d'erreur :  15  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- plus de la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de plus de 80% le nombre de lignes erronnées

In [11]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  735  soit  15  %
nombre de pdc sans doublon   :  4073


In [12]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             11
nom_enseigne - id_station_itinerance                1220
coordonneesXY - id_station_itinerance               1168
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1116
implantation_station - id_station_itinerance         478
nbre_pdc - id_station_itinerance                     796
condition_acces - id_station_itinerance               58
horaires - id_station_itinerance                      56
station_deux_roues - id_station_itinerance           454
adresse_station - coordonneesXY                      590

nombre d'enregistrements sans erreurs :  728
nombre d'enregistrements avec au moins une erreur :  3345


-----------------------
## 6 - Séparation doublons station - date de maj
- plus de 15% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de plus de 20% le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 33 opérateurs et sont liées à des causes multiples

In [13]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['etape3_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  258  soit  8  %
nombre de pdc sans doublon   :  3087


In [14]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                 909
coordonneesXY - id_station_itinerance                885
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  839
implantation_station - id_station_itinerance         275
nbre_pdc - id_station_itinerance                     580
condition_acces - id_station_itinerance               44
horaires - id_station_itinerance                      52
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                      574

nombre d'enregistrements sans erreurs :  275
nombre d'enregistrements avec au moins une erreur :  2812  soit  84  %


In [15]:
# structure des données résiduelles et nombre de valeurs
il = Sdataset(itinerance_5)
print(il.tree())

-1: root-derived (2812)
   0 : index (2812)
      1 : contact_operateur (34)
      2 : nom_enseigne (784)
      3 : coordonneesXY (822)
      4 : adresse_station (931)
      5 : id_station_itinerance (768)
         13: date_maj (62)
      6 : nom_station (893)
      7 : implantation_station (5)
      8 : nbre_pdc (16)
      9 : condition_acces (2)
      10: horaires (67)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (2812)
      14: last_modified (64)


In [16]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  6  erreurs :  10


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [17]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  2812
total des doublons à supprimer :  993
nombre de pdc avec controles ok :  27880


In [18]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [19]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [20]:
# structure des données bonnes
il = Sdataset(itinerance_ok.loc[:, mandatory])
print(il.tree())

-1: root-derived (27880)
   11: id_pdc_itinerance (27880)
      4 : id_station_itinerance (14776)
         1 : nom_enseigne (1572)
         2 : coordonneesXY (9946)
            0 : contact_operateur (119)
            3 : adresse_station (9305)
            8 : condition_acces (2)
         5 : nom_station (10034)
         6 : implantation_station (5)
         7 : nbre_pdc (28)
         9 : horaires (164)
         10: station_deux_roues (5)
         19: paiement_autre (7)
      12: puissance_nominale (67)
      13: prise_type_ef (8)
      14: prise_type_2 (8)
      15: prise_type_combo_ccs (7)
      16: prise_type_chademo (7)
      17: prise_type_autre (6)
      18: paiement_acte (8)
      20: reservation (6)
      21: accessibilite_pmr (4)
      22: restriction_gabarit (109)
      23: date_maj (512)
      24: last_modified (436)


### Indicateurs

In [21]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [22]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  77  % des écarts

support@modulo-energies.fr 729
supervision-ev.france@totalenergies.com 506
info@ionity.eu 341
assistance-commerciale@e-vadea.fr 321
pascal.lhermitte@siege27.fr 260


In [23]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, pr=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           1779
63  %


In [24]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 29 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [25]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
511,3997,support@modulo-energies.fr,Erlon 1 - 7680,"[4.025925, 49.25696]","93 Place Drouet d'Erlon, 51100 Reims",FRE10E768,Erlon 1 - 7680,Voirie,4,Accès libre,24/7,false,FRE10E7680,2023-07-23,2023-07-24T06:32:44.081000+00:00
512,3998,support@modulo-energies.fr,Erlon 1 - 7680,"[4.025925, 49.25696]","93 Place Drouet d'Erlon, 51100 Reims",FRE10E768,Erlon 1 - 7680,Voirie,4,Accès libre,24/7,false,FRE10E7681,2023-07-23,2023-07-24T06:32:44.081000+00:00
513,3999,support@modulo-energies.fr,Erlon 1 - 7680,"[4.025925, 49.25696]","93 Place Drouet d'Erlon, 51100 Reims",FRE10E768,Erlon 1 - 7680,Voirie,4,Accès libre,24/7,false,FRE10E7682,2023-07-23,2023-07-24T06:32:44.081000+00:00
514,4000,support@modulo-energies.fr,Erlon 1 - 7680,"[4.025925, 49.25696]","93 Place Drouet d'Erlon, 51100 Reims",FRE10E768,Erlon 1 - 7680,Voirie,4,Accès libre,24/7,false,FRE10E7683,2023-07-23,2023-07-24T06:32:44.081000+00:00
515,4001,support@modulo-energies.fr,Gambetta 3 - 7686,"[4.036172, 49.249946]","22 Rue Gambetta, 51100 Reims",FRE10E768,Gambetta 3 - 7686,Parking privé à usage public,4,Accès réservé,24/7,false,FRE10E7684,2023-07-23,2023-07-24T06:32:44.081000+00:00
516,4002,support@modulo-energies.fr,Gambetta 3 - 7686,"[4.036172, 49.249946]","22 Rue Gambetta, 51100 Reims",FRE10E768,Gambetta 3 - 7686,Parking privé à usage public,4,Accès réservé,24/7,false,FRE10E7685,2023-07-23,2023-07-24T06:32:44.081000+00:00
517,4003,support@modulo-energies.fr,Gambetta 3 - 7686,"[4.036172, 49.249946]","22 Rue Gambetta, 51100 Reims",FRE10E768,Gambetta 3 - 7686,Parking privé à usage public,4,Accès réservé,24/7,false,FRE10E7686,2023-07-23,2023-07-24T06:32:44.081000+00:00
518,4004,support@modulo-energies.fr,Gambetta 3 - 7686,"[4.036172, 49.249946]","22 Rue Gambetta, 51100 Reims",FRE10E768,Gambetta 3 - 7686,Parking privé à usage public,4,Accès réservé,24/7,false,FRE10E7687,2023-07-23,2023-07-24T06:32:44.081000+00:00
519,4005,support@modulo-energies.fr,Hôtel de ville 2 - 7689,"[4.03122394, 49.25696581]","5 Rue de Pouilly, 51100 Reims",FRE10E768,Hôtel de ville 2 - 7689,Parking privé à usage public,2,Accès réservé,24/7,false,FRE10E7688,2023-07-23,2023-07-24T06:32:44.081000+00:00
520,4006,support@modulo-energies.fr,Hôtel de ville 2 - 7689,"[4.03122394, 49.25696581]","5 Rue de Pouilly, 51100 Reims",FRE10E768,Hôtel de ville 2 - 7689,Parking privé à usage public,2,Accès réservé,24/7,false,FRE10E7689,2023-07-23,2023-07-24T06:32:44.081000+00:00


### Cohérence implantation_station - id_station
- 64 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [26]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
30,238,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2023-07-24T12:06:30.015000+00:00
31,239,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2023-07-24T12:06:30.015000+00:00
32,240,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2023-07-24T12:06:30.015000+00:00
33,241,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2023-07-24T12:06:30.015000+00:00
511,3997,support@modulo-energies.fr,Erlon 1 - 7680,"[4.025925, 49.25696]","93 Place Drouet d'Erlon, 51100 Reims",FRE10E768,Erlon 1 - 7680,Voirie,4,Accès libre,24/7,false,FRE10E7680,2023-07-23,2023-07-24T06:32:44.081000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,28077,klantenservice@newmotion.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE94,Shell Fenioux Est,Voirie,8,Accès libre,24/7,false,FRSHEE944,2023-07-23,2023-07-24T06:32:44.081000+00:00
2532,28078,klantenservice@newmotion.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE951,2023-07-23,2023-07-24T06:32:44.081000+00:00
2533,28079,klantenservice@newmotion.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE952,2023-07-23,2023-07-24T06:32:44.081000+00:00
2534,28080,klantenservice@newmotion.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE953,2023-07-23,2023-07-24T06:32:44.081000+00:00


### Cohérence nom_station - id_station
- 328 pdc sont associés à une station avec un nom non cohérent

In [27]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
35,246,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2023-07-24T12:06:30.015000+00:00
36,247,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2023-07-24T12:06:30.015000+00:00
37,248,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2023-07-24T12:06:30.015000+00:00
38,249,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2023-07-24T12:06:30.015000+00:00
39,250,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2023-07-24T12:06:30.015000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,23551,support@modulo-energies.fr,55 DIEUX SUR MEUSE-rue des dames-66466,"[5.427367, 49.071221]","4 Route des Dames, 55320 DIEUE SUR MEUSE",FRS55E6646,55 DIEUX SUR MEUSE-rue des dames-66466,Voirie,2,Accès réservé,24/7,false,FRS55E66467,2023-07-23,2023-07-24T06:32:44.081000+00:00
2396,23554,support@modulo-energies.fr,55 DUN SUR MEUSE-rue faubourg st Martin-66576,"[5.185284, 49.381609]","Rue du Faubourg Saint-Martin, 55110 DUN SUR MEUSE",FRS55E6657,55 DUN SUR MEUSE-rue faubourg st Martin-66576,Voirie,2,Accès réservé,24/7,false,FRS55E66576,2023-07-23,2023-07-24T06:32:44.081000+00:00
2397,23555,support@modulo-energies.fr,55 DUN SUR MEUSE-rue faubourg st Martin-66576,"[5.185284, 49.381609]","Rue du Faubourg Saint-Martin, 55110 DUN SUR MEUSE",FRS55E6657,55 DUN SUR MEUSE-rue faubourg st Martin-66576,Voirie,2,Accès réservé,24/7,false,FRS55E66577,2023-07-23,2023-07-24T06:32:44.081000+00:00
2398,23556,support@modulo-energies.fr,55 VERDUN - Rue de Jeanne d'arc - 66578,"[5.383243, 49.163453]","Place Jeanne d'Arc, 55100 Verdun",FRS55E6657,55 VERDUN - Rue de Jeanne d'arc - 66578,Voirie,2,Accès réservé,24/7,false,FRS55E66578,2023-07-23,2023-07-24T06:32:44.081000+00:00


### Cohérence adresse - coordonnées
- 440 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [28]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,102,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,24/7,FALSE,FR3R3E10000849681,2023-07-03,2023-07-24T06:33:22.097000+00:00
19,103,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,24/7,FALSE,FR3R3E10000849682,2023-07-03,2023-07-24T06:33:22.097000+00:00
20,104,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,FALSE,FR3R3E10000849691,2023-07-03,2023-07-24T06:33:22.097000+00:00
21,105,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,FALSE,FR3R3E10000849692,2023-07-03,2023-07-24T06:33:22.097000+00:00
22,106,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,FALSE,FR3R3E10000849851,2023-07-03,2023-07-24T06:33:22.097000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,33201,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2023-07-24T06:34:52.285000+00:00
3083,33210,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-07-24T06:34:53.057000+00:00
3084,33211,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-07-24T06:34:53.057000+00:00
3085,33212,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-07-24T06:34:53.057000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [29]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2282,29335,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-07-24T06:31:58.523000+00:00
2283,29336,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-07-24T06:31:58.523000+00:00
2284,29337,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-07-24T06:31:58.523000+00:00
2285,29338,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-07-24T06:31:58.523000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [30]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1338,11570,info@ionity.eu,Pamiers,"[1.641617, 43.074883]","Parc Commercial Delta-Sud ZA de la Grausette, ...",FRIONE44660,Pamiers,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIONE446604,2023-07-23,2023-07-24T06:32:44.081000+00:00
1339,11571,info@ionity.eu,"Verniolle | Super U, Parc Commercial Delta-Sud","[1.642716, 43.075236]","Rue Clément Ader, 09340 Verniolle",FRIONE44660,"Verniolle | Super U, Parc Commercial Delta-Sud",Station dédiée à la recharge rapide,1,Accès réservé,24/7,false,FRIONE446605,2023-07-23,2023-07-24T06:32:44.081000+00:00
1340,11572,info@ionity.eu,Pamiers,"[1.641617, 43.074883]","Parc Commercial Delta-Sud ZA de la Grausette, ...",FRIONE44660,Pamiers,Station dédiée à la recharge rapide,7,Accès libre,24/7,false,FRIONE446606,2023-07-23,2023-07-24T06:32:44.081000+00:00
1342,11818,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.260794257663834, 43.704066010469525]",12 Avenue Thiers 6000 NICE,FRIPKNRME,INTERPARKING - Nice Gare Thiers,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKEGTH0029,2023-06-02,2023-07-24T06:31:58.523000+00:00
1343,11871,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM001,2023-06-02,2023-07-24T06:31:58.523000+00:00
1344,11872,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM002,2023-06-02,2023-07-24T06:31:58.523000+00:00
1345,11873,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM003,2023-06-02,2023-07-24T06:31:58.523000+00:00
1346,11874,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM004,2023-06-02,2023-07-24T06:31:58.523000+00:00
1347,11875,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM005,2023-06-02,2023-07-24T06:31:58.523000+00:00
1348,11876,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM006,2023-06-02,2023-07-24T06:31:58.523000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [31]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-07-24T06:28:58.200000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-07-24T06:28:58.200000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-07-24T06:28:58.200000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-07-24T06:28:58.200000+00:00
5,11,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-07-24T06:28:58.200000+00:00
6,12,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2023-07-24T06:28:58.200000+00:00
7,13,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-07-24T06:28:58.200000+00:00
8,14,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-07-24T06:28:58.200000+00:00
9,15,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2023-07-24T06:28:58.200000+00:00
10,16,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-07-24T06:28:58.200000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [32]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
40,257,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-07-24T12:06:30.015000+00:00
41,258,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-07-24T12:06:30.015000+00:00
42,259,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-07-24T12:06:30.015000+00:00
43,260,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-07-24T12:06:30.015000+00:00
44,261,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-07-24T12:06:30.015000+00:00
45,262,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-07-24T12:06:30.015000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [33]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2132,24564,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2023-07-24,2023-07-24T06:29:27.052000+00:00
2133,24565,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2023-07-24,2023-07-24T06:29:27.052000+00:00
2134,24566,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2023-07-24,2023-07-24T06:29:27.052000+00:00
2135,24567,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2023-07-24,2023-07-24T06:29:27.052000+00:00


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [34]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
12265,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [35]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
27140,FRSDGEMBDG511,FRSDGPMBDG511,"[2.451322, 45.66523]",2
27141,FRSDGEMBDG512,FRSDGPMBDG512,"[2.451322, 45.66523]",2


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [36]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
10174,FRIENE35353A11,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
10176,FRIENE35353A21,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
10178,FRIENE35353A31,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
10180,FRIENE35353A41,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
10182,FRIENE35353A51,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [37]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PBARREOUCHEMOMORT', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18007,18688,pascal.lhermitte@siege27.fr,SIEGE265,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTD,2021-06-15,2023-07-24T06:31:06.066000+00:00
18008,18689,pascal.lhermitte@siege27.fr,SIEGE266,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTG,2021-06-15,2023-07-24T06:31:06.066000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [38]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
17628,18309,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2023-07-24T06:31:02.860000+00:00
17629,18310,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2023-07-24T06:31:02.860000+00:00


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues

In [39]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1902', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
31174,31889,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19021,2023-07-06,2023-07-24T06:31:59.221000+00:00
31175,31890,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19022,2023-07-06,2023-07-24T06:31:59.221000+00:00
31176,31891,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19023,2023-07-06,2023-07-24T06:31:59.221000+00:00
31177,31892,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19024,2023-07-06,2023-07-24T06:31:59.221000+00:00
31178,31893,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19025,2023-07-06,2023-07-24T06:31:59.221000+00:00
31179,31894,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19026,2023-07-06,2023-07-24T06:31:59.221000+00:00
31180,31895,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19027,2023-07-06,2023-07-24T06:31:59.221000+00:00


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [40]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### données d'un opérateur
- exemple Virta

In [41]:
itinerance_operateur = itinerance_5_full.loc[itinerance_5_full.contact_operateur == 'support@virta.global']
#itinerance_operateur.loc[itinerance_operateur['adresse_station - coordonneesXY'] == False, relations]
itinerance_operateur.loc[itinerance_operateur['nom_station - id_station_itinerance'] == False, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
